In [59]:
import pandas as pd
import numpy as np
import spacy as sp
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.language import Language
from keras.preprocessing.text import Tokenizer



ModuleNotFoundError: No module named 'tensorflow'

In [3]:
data = pd.read_json("/Users/lframba/Juypter/CS663-Final-Project/CS663_Project_Data.json", orient='index')
data = data

In [9]:
df = data.copy()
df

,invocation,cmd
1,"Copy loadable kernel module ""mymodule.ko"" to t...",sudo cp mymodule.ko /lib/modules/$(uname -r)/k...
2,"Display all lines containing ""IP_MROUTE"" in th...",cat /boot/config-`uname -r` | grep IP_MROUTE
3,Display current running kernel's compile-time ...,cat /boot/config-`uname -r`
4,"Find all loadable modules for current kernel, ...",find /lib/modules/`uname -r` -regex .*perf.*
5,"Look for any instance of ""HIGHMEM"" in the curr...",grep “HIGHMEM” /boot/config-`uname -r`
...,...,...
10343,using exec in find command to dispaly the sear...,find . ... -exec cat {} \; -exec echo \;
10344,verbosely create intermediate directoriy tmp a...,mkdir -pv /tmp/boostinst
10345,view the manual page of find,man find
10346,"wait 2 seconds and then print ""hello""","echo ""hello `sleep 2 &`"""


In [55]:
nlp = sp.load('en_core_web_sm')

@Language.component('clean_inv')
def clean_inv(doc):
    # Lowercase
    doc = [token.lower_ for token in doc]
    # # Punctuation + special chars
    # tokens = [token for token in tokens if not token.is_punct and not token.is_space and not token.is_digit and not token.like_num and not token.is_currency and not token.is_stop]
    # # Stop words
    # tokens = [token for token in tokens if not token in STOP_WORDS]
    return doc

#nlp.add_pipe('clean_inv', name='clean_inv', last=True)

def tokenize_inv(inv):
    doc = nlp(inv)
    tokens = [token.text for token in doc]
    return tokens


In [56]:

df['inv_tokens'] = df['invocation'].apply(tokenize_inv)
df

,invocation,cmd,inv_tokens
1,"Copy loadable kernel module ""mymodule.ko"" to t...",sudo cp mymodule.ko /lib/modules/$(uname -r)/k...,"[Copy, loadable, kernel, module, "", mymodule.k..."
2,"Display all lines containing ""IP_MROUTE"" in th...",cat /boot/config-`uname -r` | grep IP_MROUTE,"[Display, all, lines, containing, "", IP_MROUTE..."
3,Display current running kernel's compile-time ...,cat /boot/config-`uname -r`,"[Display, current, running, kernel, 's, compil..."
4,"Find all loadable modules for current kernel, ...",find /lib/modules/`uname -r` -regex .*perf.*,"[Find, all, loadable, modules, for, current, k..."
5,"Look for any instance of ""HIGHMEM"" in the curr...",grep “HIGHMEM” /boot/config-`uname -r`,"[Look, for, any, instance, of, "", HIGHMEM, "", ..."
...,...,...,...
10343,using exec in find command to dispaly the sear...,find . ... -exec cat {} \; -exec echo \;,"[using, exec, in, find, command, to, dispaly, ..."
10344,verbosely create intermediate directoriy tmp a...,mkdir -pv /tmp/boostinst,"[verbosely, create, intermediate, directoriy, ..."
10345,view the manual page of find,man find,"[view, the, manual, page, of, find]"
10346,"wait 2 seconds and then print ""hello""","echo ""hello `sleep 2 &`""","[wait, 2, seconds, and, then, print, "", hello, ""]"


ValueError: [E005] Pipeline component 'clean_inv' returned <class 'list'> instead of a Doc. If you're using a custom component, maybe you forgot to return the processed Doc?